In [1]:
#Load the packages
import torch
import torch.nn as nn
from lightning.pytorch import Trainer #https://lightning.ai/docs/pytorch/stable/common/trainer.html
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
from maldi_zsl_edit.data import MALDITOFDataModule
from maldi_zsl_edit.models import ZSLClassifier
import h5py
import numpy as np

In [2]:
#What to tune
batch_size = 16, 32 ,64
dim_emb = 512,788,1014
lr=1e-4,1e-6
dropout = 0.2,0.5
k_n = 3,5,7
c_n = 0,64,128

In [3]:
#Load the data set
dm = MALDITOFDataModule( #Personalized lightning data modules
    "../Data/zsl_binned_new.h5t", #The old has problems on split
    zsl_mode = True, # False: multi-class CLF, True: ZSL
    split_index = 0, # independent train-val-test split numbered 0-9
    batch_size = 32, # important hyperparameter
    n_workers = 2, # you can leave this always if you are not CPU limited
    in_memory = True, # you can leave this always if memory is no problem
    )

dm.setup(None)
#batch = next(iter(dm.train_dataloader()))
#batch.keys()

## Training

In [4]:
#Now there should be a batch instance ["seq_ohe]", replace the batch["seq"] with it in the models file
n_species = 160 #batch['strain'].shape[0] #Number the seq considered for the train, #The batch should be 623 (463 of training and 160 of val, the rest 165 are on test)
t_species = 463
#To add the correct number of n_species you need to correct the sequences seen per batch
dim_emb = 1024
model = ZSLClassifier(
    mlp_kwargs = { #specify the parameters to buld the MLP ()
        'n_inputs' : 6000, #Bins of the spectra
        'emb_dim' : dim_emb, #This is the output of the branch
        'layer_dims': [256, 128],
        'layer_or_batchnorm' : "layer",
        'dropout' : 0.2,
    },
    cnn_kwargs= { #specify the parameters to buld the CNN ()
        'vocab_size' : 6, #Number of words, in this case is 5 as (A,T,C,G,-)
        'emb_dim' : dim_emb, #This is the output of the branch
        'conv_sizes' : [64, 128], #[32, 64, 128] Out chanels of the convolutions #On the nlp mode the first is an embeding dimension
        'hidden_sizes' : [0], #MLP: [512, 256]. If [0] then goes directly from conv to embeding layer
        #IMPORTANT: The models for classification have first the convolution and then a MLP, consider to also add the MLP in the model
        #Note: The first hidden state is the embedding dim of the seq language processing and need to be optimized
        #Note2: The last is the embedding dim for the shared space and score function
        'blocks_per_stage' : 2, #How many residual blocks are applied before the pooling
        'kernel_size' : 7,
        #Stride?
        #Max average or non?
        'dropout' : 0.2,
        'nlp' : False #Move directly to the branch
    },
    n_classes = n_species,
    t_classes = t_species,
    lr=1e-4, # important to tune
    weight_decay=0, # this you can keep constant
    lr_decay_factor=1.00, # this you can keep constant
    warmup_steps=250, # this you can keep constant
    #nlp = False #Try
)

In [5]:
#Save and monitor training with tensor board
from datetime import datetime
timenow = datetime.now()
strtime = timenow.strftime("%Y-%m-%d_%H-%M-%S")

val_ckpt = ModelCheckpoint(monitor="val_acc", mode="max")
callbacks = [val_ckpt, EarlyStopping(monitor="val_acc", patience=10, mode="max")]
logger = TensorBoardLogger("../logs_folder", name="zsl_train_try2", version=strtime) # Ctrl+Shift+P # Main folder where the training is saved and the name for the training

#Training specification
trainer = Trainer(
    max_epochs = 100, 
    accelerator='gpu', 
    strategy='auto',
    callbacks=callbacks,
    logger=logger,
    devices=[0]) #You can define epochs and training devices (look on documentation)

/home/jorge/anaconda3/envs/Thesis/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
#Pre-save the model
#sure = True
#if sure:
#    torch.save(model, f'../SavedModels/ZSLmodel{strtime}.pth')
#    print(f"Saved as ZSLmodel{strtime}.pth")
#model.load_from_checkpoint('../logs_folder/zsl_train_try2/2024-07-16_08-12-50/checkpoints/epoch=25-step=27404.ckpt') #load model from if needed

#Start training
trainer.fit(model, dm.train_dataloader(), dm.val_dataloader()) #Important: normally you can use only dm, but here we specify as the dim of a are different for train and val 
#Note: The model object specify what is considered an input values and what is considered an input/output value during the training on the training step method

/home/jorge/anaconda3/envs/Thesis/lib/python3.11/site-packages/torch/cuda/__init__.py:190: UserWarning: 
    Found GPU1 NVIDIA Tesla K40c which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(
2024-07-19 06:29:25.362198: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-19 06:29:26.268738: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name              | Type               | Params
---------------------------------------------------------
0 | spectrum_embedder | MLPEmbedding       | 1.9 M 
1 | seq_embedder      | CNNEmbeddin

Sanity Checking: 0it [00:00, ?it/s]

/home/jorge/anaconda3/envs/Thesis/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:01<00:00,  1.98it/s]
Epoch 0 - Train loss: NA, Train accu: 0.0
Epoch 0 - Val loss: 6.65458869934082, Val accu: 0.0

                                                                           

/home/jorge/anaconda3/envs/Thesis/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|██████████| 527/527 [05:18<00:00,  1.65it/s, v_num=9-25]
Epoch 0 - Train loss: 8.811540603637695, Train accu: 0.0017789374105632305
Epoch 0 - Val loss: 5.0762457847595215, Val accu: 0.012768817134201527

Epoch 1: 100%|██████████| 527/527 [05:19<00:00,  1.65it/s, v_num=9-25]
Epoch 1 - Train loss: 6.4172563552856445, Train accu: 0.002964895684272051
Epoch 1 - Val loss: 5.082064151763916, Val accu: 0.009072580374777317

Epoch 2: 100%|██████████| 527/527 [05:19<00:00,  1.65it/s, v_num=9-25]
Epoch 2 - Train loss: 6.325634002685547, Train accu: 0.0016603415133431554
Epoch 2 - Val loss: 5.075387001037598, Val accu: 0.010080644860863686

Epoch 3: 100%|██████████| 527/527 [05:19<00:00,  1.65it/s, v_num=9-25]
Epoch 3 - Train loss: 6.276715278625488, Train accu: 0.0024312143214046955
Epoch 3 - Val loss: 5.0764851570129395, Val accu: 0.005376344081014395

Epoch 4: 100%|██████████| 527/527 [05:19<00:00,  1.65it/s, v_num=9-25]
Epoch 4 - Train loss: 6.241939544677734, Train accu: 0.0027

In [7]:
#timenow = datetime.now()
#traintime = timenow.strftime("%Y-%m-%d_%H-%M-%S") - strtime
#print(f"The model lasted {traintime} to train")

#Save the model also at the end of the training
sure = True
if sure:
    torch.save(model, f'../SavedModels/ZSLmodel{strtime}.pth')
    print(f"Saved as ZSLmodel{strtime}.pth")


Saved as ZSLmodel2024-07-19_06-29-25.pth


# Test model

In [8]:
#Load the model
sure = False
if sure:
    model = torch.load('../SavedModels/ZSLmodel2024-07-13_02-56-19.pth')

## Get predictions

In [9]:
minibatch = next(iter(dm.val_dataloader()))
print(minibatch["intensity"].shape)
print(minibatch["seq_ohe"].shape) #The batch should be 623 (463 of training and 160 of val, the rest 165 are on test)

torch.Size([32, 6000])
torch.Size([160, 6, 2236])


In [10]:
#Info of the data set
h5spectra = h5py.File("../Data/zsl_binned_new.h5t", "r") # The old version has problems on the split
h5spectra.visititems(print)
#There are train: 463, val :160 and test: 165 
torch.cuda.empty_cache() 

0 <HDF5 group "/0" (11 members)>
0/intensity <HDF5 dataset "intensity": shape (29105, 6000), type "<f4">
0/split_0 <HDF5 dataset "split_0": shape (29105,), type "|S11">
0/split_1 <HDF5 dataset "split_1": shape (29105,), type "|S11">
0/split_2 <HDF5 dataset "split_2": shape (29105,), type "|S11">
0/split_3 <HDF5 dataset "split_3": shape (29105,), type "|S11">
0/split_4 <HDF5 dataset "split_4": shape (29105,), type "|S11">
0/split_5 <HDF5 dataset "split_5": shape (29105,), type "|S11">
0/split_6 <HDF5 dataset "split_6": shape (29105,), type "|S11">
0/split_7 <HDF5 dataset "split_7": shape (29105,), type "|S11">
0/split_8 <HDF5 dataset "split_8": shape (29105,), type "|S11">
0/split_9 <HDF5 dataset "split_9": shape (29105,), type "|S11">
1 <HDF5 group "/1" (4 members)>
1/strain_names <HDF5 dataset "strain_names": shape (788,), type "|S124">
1/strain_seq <HDF5 dataset "strain_seq": shape (788,), type "|S1676">
1/strain_seq_aligned <HDF5 dataset "strain_seq_aligned": shape (788, 2236), type

In [11]:
#Separate the predictions to look their individual accuracies
ev_species = {}
ev_species[b'train'] = []
ev_species[b'val_geni'] = []
ev_species[b'val_spec'] = []
ev_species[b'val_strain'] = []
study = [b'train',b'val_geni',b'val_spec',b'val_strain']
i = 0
for label in h5spectra["0"]["split_0"]:
    if label in study:
        a = h5spectra["central"][i]
        b = np.where(a == True)[0][0]
        ev_species[label].append(b)
    i+=1

In [12]:
#Manual calculation of the predictions
import torch.nn as nn
import torch
y_pred = torch.empty((0,160)) #the second is the number of species #Change to 788 or 463 for val vs train
y_real= []
with torch.no_grad():
    for minibatch in iter(dm.val_dataloader()): #On the split said if train, val, etc, 
        y_hat = model(minibatch)
        y_pred = torch.cat((y_pred,y_hat),dim=0)
        y_real+= list(minibatch['strain'])
print(y_pred.shape) #(batch size, total possible species)
y_pred

torch.Size([5941, 160])


tensor([[-0.5757, -0.1364, -0.5885,  ..., -0.7908, -0.4338, -0.3347],
        [ 3.7039,  4.4235,  3.1040,  ...,  3.2766,  3.4193,  3.7365],
        [-2.5605, -2.2706, -2.8003,  ..., -2.6501, -2.4468, -2.4962],
        ...,
        [12.4245, 11.9488, 11.1080,  ..., 11.5319, 11.6720, 12.1082],
        [-0.4275, -0.7524, -1.0025,  ..., -1.0437, -0.7795, -0.7447],
        [ 1.9765,  1.3395,  1.0238,  ...,  0.8340,  1.4872,  1.7841]])

## Labels for Multilevel evaluation

In [13]:
pred_ind = torch.argmax(y_pred, axis=1)
real_ind = y_real
levels = ["Family", "Genus", "Species", "Strain"]
granularity_lvl = len(levels) 

In [14]:
filos = minibatch["seq_names"]

In [15]:
#Get the multilevel predictions, consider how the data is encoded (genus, species, strain)
ml_real = []
ml_pred = []
for i in range(len(y_real)):
  #for real:
  s_real = filos[real_ind[i]].split(";")
  ml_real.append(s_real)
  #for pred:
  s_pred = filos[pred_ind[i]].split(";")
  ml_pred.append(s_pred)

In [16]:
#Get them on the right format
import numpy as np
ml_real = np.array(ml_real).T
ml_pred = np.array(ml_pred).T
#List for better iteratation
ml_reals = ml_real.tolist()
ml_preds = ml_pred.tolist()

In [17]:
#Get all the possible multilevel labels
ml_level = []
for i in range(len(filos)):
  s_level = filos[i].split(";")
  ml_level.append(s_level)
ml_level = np.array(ml_level).T
ml_levels = ml_level.tolist()

In [18]:
#Total number of labels
for i in range(granularity_lvl):
    n = len(list(set(ml_levels[i])))
    print(f"For {levels[i]} there are {n} different labels")

For Family there are 33 different labels
For Genus there are 52 different labels
For Species there are 152 different labels
For Strain there are 160 different labels


## Accuracy evaluation

In [19]:
from sklearn.metrics import accuracy_score #There is also a torch version, consider it
from sklearn.preprocessing import LabelEncoder
from torchmetrics import Accuracy

#Create a accuracy evaluator
def accu_score(y_true, y_pred, level_lab):
    label_encoder = LabelEncoder()
    y_true_encoded = label_encoder.fit_transform(level_lab)
    y_true_encoded = label_encoder.transform(y_true)
    y_pred_encoded = label_encoder.transform(y_pred)

    #Using sklearn
    #accu = accuracy_score(y_true_encoded, y_pred_encoded, normalize=True) #The normalize True = number of correct predictions, False = fraction of correct predictions
    
    #Using torch
    accu = Accuracy(task="multiclass", num_classes=len(set(level_lab))) 
    accu = accu(torch.tensor(y_pred_encoded), torch.tensor(y_true_encoded))
    
    return accu

In [20]:
# run accu for each level of complexity
accu_levels = []
for level in range(granularity_lvl ):
  accu_levels.append(accu_score(ml_reals[level], ml_preds[level], ml_levels[level]))

In [21]:
# see the results
for i in range(granularity_lvl ):
  print(f"For the level {levels[i]} the accu score is: {accu_levels[i]}") 

For the level Family the accu score is: 0.09089379012584686
For the level Genus the accu score is: 0.05201144516468048
For the level Species the accu score is: 0.005722942296415567
For the level Strain the accu score is: 0.0055546206422150135


## F1 evaluation

In [22]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from torchmetrics import F1Score

#Create an F1 evaluator
def f1_macro_score(y_true, y_pred, level_lab): #micro average is basically accuracy
    label_encoder = LabelEncoder()
    y_true_encoded = label_encoder.fit_transform(level_lab)
    y_true_encoded = label_encoder.transform(y_true)
    y_pred_encoded = label_encoder.transform(y_pred)

    #Using sklearn
    #f1_scores = f1_score(y_true_encoded, y_pred_encoded, average=None)
    #macro_f1 = sum(f1_scores) / len(f1_scores)

    #Using torch
    macro_f1 = F1Score(task="multiclass", num_classes=len(set(level_lab)), average='macro') 
    macro_f1 = macro_f1(torch.tensor(y_pred_encoded), torch.tensor(y_true_encoded))

    return macro_f1

In [23]:
# run f1_macro_score for each level of complexity
F1_levels = []
for level in range(granularity_lvl ):
  F1_levels.append(f1_macro_score(ml_reals[level], ml_preds[level], ml_levels[level]))

In [24]:
# see the results
for i in range(granularity_lvl ):
  print(f"For the level {levels[i]} the F1 score is: {F1_levels[i]}") #The predictions are no the same as the output, maybe F1 is not used there

For the level Family the F1 score is: 0.037777528166770935
For the level Genus the F1 score is: 0.02623894438147545
For the level Species the F1 score is: 0.006067595444619656
For the level Strain the F1 score is: 0.005674286745488644
